In [1]:
import sys
import lucene
import os
import utils
import book_processing
import pandas as pd
import numpy as np
import re
from lucene import *

In [2]:
lucene.initVM()

In [3]:
data_test = pd.read_csv("data/training_set.tsv", sep = '\t')
data_val = pd.read_csv("data/validation_set.tsv", sep = '\t')

In [4]:
def getDocument(fname):
    doc = Document()
    doc.add(Field('filename', os.path.split(fname)[-1], Field.Store.YES, Field.Index.NOT_ANALYZED))
    doc.add(Field('content', open(fname).read(), Field.Store.YES, Field.Index.ANALYZED))
    return doc

def indexFile(fname, writer):
    doc = getDocument(fname)
    writer.addDocument(doc)

def indexDirectory(dir_path, writer):
    for fname in os.listdir(dir_path):
        indexFile(os.path.join(dir_path, fname), writer)
    return writer.numDocs()

def indexDictionary(d, writer):
    for k, v in d.iteritems():
        doc = Document()
        doc.add(Field('filename', k, Field.Store.YES, Field.Index.NOT_ANALYZED))
        doc.add(Field('content', v, Field.Store.YES, Field.Index.ANALYZED))
        writer.addDocument(doc)
    return writer.numDocs()

In [5]:
#index wiki articles based on ck 12 topics
#analyzer = StandardAnalyzer(Version.LUCENE_30)
analyzer = SnowballAnalyzer(Version.LUCENE_30, "English", StandardAnalyzer.STOP_WORDS_SET)
writer = IndexWriter(SimpleFSDirectory(File("data/index/wiki_ck12")), analyzer, True, IndexWriter.MaxFieldLength.UNLIMITED)
indexDirectory('data/wiki_data', writer)

2139

In [6]:
writer.close()

In [7]:
#index topics from ck12 book (document is text between h1 tags)
dir_name = 'data/ck12_book/OEBPS'
docs = {}
html_paths = [os.path.join(dir_name,  str(i+1) + '.html') for i in range(124)]
for f_name in html_paths:
    docs.update(book_processing.get_h1_text(open(f_name).read()))

#analyzer = StandardAnalyzer(Version.LUCENE_30)
analyzer = SnowballAnalyzer(Version.LUCENE_30, "English", StandardAnalyzer.STOP_WORDS_SET)
writer = IndexWriter(SimpleFSDirectory(File("data/index/ck12_books_topics")), analyzer, True, IndexWriter.MaxFieldLength.UNLIMITED)
indexDictionary(docs, writer)

1723

In [8]:
writer.close()

In [13]:
#index paragraphs from ck12 book (document is text between any h tags)
dir_name = 'data/ck12_book/OEBPS'
docs = {}
html_paths = [os.path.join(dir_name,  str(i+1) + '.html') for i in range(124)]
for f_name in html_paths:
    docs.update(book_processing.get_h_all_text(open(f_name).read()))

#analyzer = StandardAnalyzer(Version.LUCENE_30)
analyzer = SnowballAnalyzer(Version.LUCENE_30, "English", StandardAnalyzer.STOP_WORDS_SET)
writer = IndexWriter(SimpleFSDirectory(File("data/index/ck12_books_paragraphs")), analyzer, True, IndexWriter.MaxFieldLength.UNLIMITED)
indexDictionary(docs, writer)

10040

In [14]:
writer.close()

In [15]:
res = {}
MAX = 100
docs_per_q = range(1,20)

#analyzer = StandardAnalyzer(Version.LUCENE_30)
analyzer = SnowballAnalyzer(Version.LUCENE_30, "English", StandardAnalyzer.STOP_WORDS_SET)
reader = IndexReader.open(SimpleFSDirectory(File("data/index/ck12_books_paragraphs")))
searcher = IndexSearcher(reader)

for index, row in data_test.iterrows():
        
    queries = [row['answerA'], row['answerB'], row['answerC'], row['answerD']]
    queries = [row['question'] + ' ' + q  for q in queries]
        
    scores = {}
    for q in queries:
        query = QueryParser(Version.LUCENE_30, "content", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
        #query = QueryParser(Version.LUCENE_30, "content", analyzer).parse(re.sub("[/^]", "\^", q))
        hits = searcher.search(query, MAX)
        doc_importance = [hit.score for hit in hits.scoreDocs]
        for n in docs_per_q:
            scores.setdefault(n, [])
            scores[n].append(sum(doc_importance[:n]))
      
    for n in docs_per_q:
        res.setdefault(n, [])
        res[n].append(['A','B','C','D'][np.argmax(scores[n])])

In [16]:
for k, v in sorted(res.items(), key=lambda x: x[0]):
    print k, 1. * sum(data_test['correctAnswer'] == v) / len(v)

1 0.42
2 0.4308
3 0.4412
4 0.444
5 0.4404
6 0.44
7 0.4368
8 0.4372
9 0.4396
10 0.4408
11 0.4376
12 0.4364
13 0.4384
14 0.4356
15 0.4348
16 0.4336
17 0.4332
18 0.4296
19 0.4276


In [75]:
#alternative prediction, first get top scored questions and then calculate scores for answers
res = {}
MAX = 30
docs_per_q = range(1,20)

#analyzer = StandardAnalyzer(Version.LUCENE_30)
analyzer = SnowballAnalyzer(Version.LUCENE_30, "English", StandardAnalyzer.STOP_WORDS_SET)
reader = IndexReader.open(SimpleFSDirectory(File("data/index/ck12_books_paragraphs")))
searcher = IndexSearcher(reader)

for index, row in data_val.iterrows():
    q =  row['question']
    query = QueryParser(Version.LUCENE_30, "content", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
    hits = searcher.search(query, MAX)
    
    sc_A = []
    sc_B = []
    sc_C = []
    sc_D = []
    
    qA = QueryParser(Version.LUCENE_30, "content", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", row['answerA']))
    qB = QueryParser(Version.LUCENE_30, "content", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", row['answerB']))
    qC = QueryParser(Version.LUCENE_30, "content", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", row['answerC']))
    qD = QueryParser(Version.LUCENE_30, "content", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", row['answerD']))
    
    
    for hit in hits.scoreDocs:
        sc_A.append(searcher.explain(qA, hit.doc).getValue())
        sc_B.append(searcher.explain(qB, hit.doc).getValue())
        sc_C.append(searcher.explain(qC, hit.doc).getValue())
        sc_D.append(searcher.explain(qD, hit.doc).getValue())
    for n in docs_per_q:
        res.setdefault(n, [])
        res[n].append(['A','B','C','D'][np.argmax([sum(sc_A[:n]), sum(sc_B[:n]), sum(sc_C[:n]), sum(sc_D[:n])])])

In [74]:
for k, v in sorted(res.items(), key=lambda x: x[0]):
    print k, 1. * sum(data_test['correctAnswer'] == v) / len(v)

1 0.3888
2 0.4064
3 0.428
4 0.4332
5 0.4388
6 0.4428
7 0.4352
8 0.4416
9 0.44
10 0.4336
11 0.43
12 0.43
13 0.428
14 0.4296
15 0.4272
16 0.4176
17 0.4156
18 0.416
19 0.414
20 0.4124
21 0.4088
22 0.4084
23 0.4064
24 0.4072
25 0.4052
26 0.4004
27 0.4004
28 0.3984
29 0.3972
30 0.398
31 0.398
32 0.398
33 0.398
34 0.398
35 0.398
36 0.398
37 0.398
38 0.398
39 0.398
40 0.398
41 0.398
42 0.398
43 0.398
44 0.398
45 0.398
46 0.398
47 0.398
48 0.398
49 0.398
50 0.398
51 0.398
52 0.398
53 0.398
54 0.398
55 0.398
56 0.398
57 0.398
58 0.398
59 0.398
60 0.398
61 0.398
62 0.398
63 0.398
64 0.398
65 0.398
66 0.398
67 0.398
68 0.398
69 0.398
70 0.398
71 0.398
72 0.398
73 0.398
74 0.398
75 0.398
76 0.398
77 0.398
78 0.398
79 0.398
80 0.398
81 0.398
82 0.398
83 0.398
84 0.398
85 0.398
86 0.398
87 0.398
88 0.398
89 0.398
90 0.398
91 0.398
92 0.398
93 0.398
94 0.398
95 0.398
96 0.398
97 0.398
98 0.398
99 0.398


In [76]:
#save result
pd.DataFrame({'id': list(data_val['id']), 'correctAnswer': res[8]})[['id', 'correctAnswer']].to_csv("prediction_ck12_h1_8_lucene_snowball_q_first.csv", index = False)